In [1]:
!pip install langchain-community
!pip install langchain_groq
!pip install PyPDF2
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.1 MB/s eta 0:00:00:00:0100:01


In [3]:
from langchain_groq import ChatGroq
import PyPDF2 as pdf
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
import langchain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [4]:
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    groq_api_key = "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",
    temperature = 0
)

In [5]:
def input_pdf_text(uploaded_pdf):
    with open(uploaded_pdf, "rb") as file:
        reader = pdf.PdfReader(file)
        text = ""
        for page in reader.pages:
            extracted = page.extract_text()
            if extracted:
                text += extracted
        return text

In [6]:
texts = input_pdf_text("/kaggle/input/ahkampdf/ahkam.pdf")

In [7]:
texts[:1000]

'اقسام نمازها \n نمازهای واجب و مستحب  \nنمازهای واجب: \n      نمازهای یومی ه  \n      نماز طواف خانه ی خدا که پس از طواف خانه ی کعبه گزارده می شود. \n      نماز آیات که در هنگام پدید آمدن خورشید و ماه گرفتگی و زلزله و امثال آنها خوانده می شود. \n      نماز میت که بر مرده خوانده می شود. \n      نماز قضای پدر و مادر که بر پسر بزرگتر واجب است . \n      نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است. \nنمازهای مستحب، مثل نافله های شبانه روزی. \nتوجه: \n نماز گویند و از میان نمازهای نافله، نافله  »نافله« های مستحبی بسیار است و آنها راهای شبانه  روزی بیشتر  \nسفارش شده است. \nنوافل یومیه  \n۱ .با هر یک از نمازهای پنجگانه ی شبانه روزی یک نماز مستحبی هست که آن را نافله ی آن نماز گویند.  \nخواندن این نافله  ها بسیار مهم است و ثواب و اجر زیاد برای آن ذکر شده است. عالوه بر اینها مستحب است  \nدر ثلث آخر شب، نافله ی شب بخواند. این نماز هم دارای خواص معنوی بسیار است و شایسته است بر آن  \nمواظبت شود. \n۲ .نافله هایشبانهروزی  \n      ۱ .نافله ی ظهر: هشت رکعت قبل  از نماز ظهر. \n

In [8]:
re_texts = re.sub("\s+", " ", texts)

In [9]:
re_texts[:1000]

'اقسام نمازها نمازهای واجب و مستحب نمازهای واجب: نمازهای یومی ه نماز طواف خانه ی خدا که پس از طواف خانه ی کعبه گزارده می شود. نماز آیات که در هنگام پدید آمدن خورشید و ماه گرفتگی و زلزله و امثال آنها خوانده می شود. نماز میت که بر مرده خوانده می شود. نماز قضای پدر و مادر که بر پسر بزرگتر واجب است . نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است. نمازهای مستحب، مثل نافله های شبانه روزی. توجه: نماز گویند و از میان نمازهای نافله، نافله »نافله« های مستحبی بسیار است و آنها راهای شبانه روزی بیشتر سفارش شده است. نوافل یومیه ۱ .با هر یک از نمازهای پنجگانه ی شبانه روزی یک نماز مستحبی هست که آن را نافله ی آن نماز گویند. خواندن این نافله ها بسیار مهم است و ثواب و اجر زیاد برای آن ذکر شده است. عالوه بر اینها مستحب است در ثلث آخر شب، نافله ی شب بخواند. این نماز هم دارای خواص معنوی بسیار است و شایسته است بر آن مواظبت شود. ۲ .نافله هایشبانهروزی ۱ .نافله ی ظهر: هشت رکعت قبل از نماز ظهر. ۲ .نافله ی عصر: هشت رکعت قبل از نماز عصر. ۳ .نافله ی مغرب: چهار رکعت بعد از نماز مغرب. ۴ .نافله 

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],
    chunk_size = 1500,
    chunk_overlap = 150
)

chunks = text_splitter.split_text(re_texts)

In [20]:
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 17


In [12]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/tmp/ipykernel_36/4241498436.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
2025-08-15 07:08:14.676550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755241694.823390      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755241694.867620      36 cuda_blas.cc:1418] Unable to register cuBLAS f

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
documents = [Document(page_content = chunk, metadata = {"source": f"chunk_{i}"}) for i, chunk in enumerate(chunks)]

In [22]:
vector_db = FAISS.from_documents(documents, embedding = embeddings)

In [23]:
retriever = vector_db.as_retriever()

In [29]:
chain = RetrievalQAWithSourcesChain.from_llm(
    llm = llm,
    retriever = retriever,
    verbose = False
)

In [25]:
chunks[0]

'اقسام نمازها نمازهای واجب و مستحب نمازهای واجب: نمازهای یومی ه نماز طواف خانه ی خدا که پس از طواف خانه ی کعبه گزارده می شود. نماز آیات که در هنگام پدید آمدن خورشید و ماه گرفتگی و زلزله و امثال آنها خوانده می شود. نماز میت که بر مرده خوانده می شود. نماز قضای پدر و مادر که بر پسر بزرگتر واجب است . نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است. نمازهای مستحب، مثل نافله های شبانه روزی. توجه: نماز گویند و از میان نمازهای نافله، نافله »نافله« های مستحبی بسیار است و آنها راهای شبانه روزی بیشتر سفارش شده است. نوافل یومیه ۱ .با هر یک از نمازهای پنجگانه ی شبانه روزی یک نماز مستحبی هست که آن را نافله ی آن نماز گویند. خواندن این نافله ها بسیار مهم است و ثواب و اجر زیاد برای آن ذکر شده است. عالوه بر اینها مستحب است در ثلث آخر شب، نافله ی شب بخواند. این نماز هم دارای خواص معنوی بسیار است و شایسته است بر آن مواظبت شود. ۲ .نافله هایشبانهروزی ۱ .نافله ی ظهر: هشت رکعت قبل از نماز ظهر. ۲ .نافله ی عصر: هشت رکعت قبل از نماز عصر. ۳ .نافله ی مغرب: چهار رکعت بعد از نماز مغرب. ۴ .نافله 

In [37]:
def generate_response(query):
    return chain(query)

In [38]:
response = generate_response("اقسام نمازهای واجب و مستحب را نام ببر")

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "اقسام نمازهای واجب و مستحب را نام ببر"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "کسی که آشنا به استفاده از آنها است) و چه از راه های دیگر، و اگر نتوانست اطمینان پیدا کند به هر طرف که گمان بیشتر مییابد، نماز بخواند . ۳ .کسی که هیچ راهی برای یافتن جهت قبله ندارد و گمانش هم به هیچ طرف نمی رود بنابر احتیاط (واجب) باید به چهار طرف نماز بخواند، و اگر برای چهار نماز وقت ندارد به هر اندازه که وقت دارد، نماز بخواند . ۴ .کسی که یقین به قبله ندارد در بقیه ی کارهایی که باید رو به قبله انجام شود، مانند ذبح حیوانات و غیره، باید به گمان خود عمل کند و اگر به هیچ طرف گمان بیشتر ندارد، به هر طرف انجام دهد صحیح است. توجه: اعتماد بر شاخص یا قبله نما در تعیین

In [41]:
print(response['answer'])

نمازهای واجب و مستحب به شرح زیر هستند:
نمازهای واجب: 
- نمازهای یومی 
- نماز طواف خانه ی خدا 
- نماز آیات 
- نماز میت 
- نماز قضای پدر و مادر 
- نمازی که به واسطه ی نذر و عهد و قسم یا به واسطه ی اجاره واجب شده است.
نمازهای مستحب، مثل نافله های شبانه روزی.



In [44]:
response = generate_response("مقدار پوشش در نماز برای مردان باید چگونه باشد؟ به زبان فارسی پاسخ بده")

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "مقدار پوشش در نماز برای مردان باید چگونه باشد؟ به زبان فارسی پاسخ بده"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "احتیاط واجب آن است که بر بام کعبه نماز نخواند . نماز خواندن بر سجاده یی که شکلهایی روی آن رسم شده و یا بر مهری که دارای نقش است فی نفسه اشکال ندارد، ولی اگر به گونه یی باشد که بهانه به دست کسانی دهد که تهمت به شیعه می زنند، تولید آن و نماز خواندن بر آن جایز نیست، و همچنین اگر موجب تفرق حواس و از بین رفتن حضور قلب در نماز شود کراهت دارد. احکام قبله ۱ .مسلمان باید نماز را رو به سمت کعبه بگزارد که به این اعتبار به می »قبله« آنگویند. البته برای کسانی که از آن دورند محاذات حقیقی میسّر نیست، و همین اندازه که بگویند رو به قبله نماز می

In [45]:
response = chain("شرایط لباس نمازگزار را بگو")

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "شرایط لباس نمازگزار را بگو"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "آن نماز خواند . ج) آن که لباسهای کوچک نمازگزار، مانند جوراب که با آنها نمیتوان عورت را پوشاند نجس باش د ۱ .اگر لباس های کوچک نمازگزار، مانند جوراب و دستکش و عرقچین که با آنها نمی توان عورت را پوشانید، و نیز انگشتر و دستبند و امثال آن با نجاستی تماس پیدا کند و نجس شود نماز خواندن با آن اشکال ندارد. ۲ .اشیایی از قبیل دستمال و کلید و کیف و چاقو که همراه انسان است اگر نجس باشد، در صورتی که نتوان عورت را با آن پوشاند، نماز اشکال ندارد. د) آن که ناچار است با بدن یا لباس نجس نماز بخواند کسی که به خاطر سرما یا نداشتن آب و امثال اینها ناچار شود که با بدن یا لباس نجس نماز بخواند،

In [46]:
print(response['answer'])

شرایط لباس نمازگزار عبارت است از: 
پاک بودن، مباح بودن (غصبی نبودن)، عدم ساخت از اجزای مردار، عدم ساخت از حیوان حرام گوشت، عدم بافت از طلا (برای مردان)، و عدم ساخت از ابریشم خالص (برای مردان).



In [48]:
response = chain("وقت نماز ظهر کی است؟ به زبان فارسی پاسخ بده")

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "وقت نماز ظهر کی است؟ به زبان فارسی پاسخ بده"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "عصر تا غروب آفتاب وقت باقی مانده است. وقت نماز عصر وقت نماز عصر از هنگامی است که به قدر خواندن نماز ظهر، از ظهر گذشته تا غروب آفتاب. وقت نماز مغرب وقت نماز مغرب از هنگامی است که سرخی بعد از غروب آفتاب که از طرف مشرق باال می آید زائل شود تا هنگامی که فقط به قدر نماز عشا تا نیمه شب مانده است. توجه: فاصله ی زمانی بین غروب خورشید و زوال حمره ی مشرقیه (از بین رفتن سرخی طرف مشرق بعد از غروب آفتاب) با اختالف فصل های سال تفاوت می کند . وقت نماز عشا وقت نماز عشا از هنگامی است که به قدر خواندن نماز مغرب از اول وقت گذشته است تا نیمه شب. توجه: احتیاط (واجب) آن است 

In [49]:
print(response['answer'])

وقت نماز ظهر از اول ظهر (یعنی هنگامی که سایه ی هر چیزی پس از آن که بر اثر باال آمدن آفتاب کامالً کوتاه شد مجدداً به طرف مشرق شروع به زیاد شدن کند) است تا هنگامی که فقط به قدر نماز عصر تا غروب آفتاب وقت باقی مانده است.

